In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory
unzip /tmp/openpayments_2023.zip -d /tmp/openpayments_2023

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0   142M      0  0:00:05  0:00:05 --:--:--  167M


Archive:  /tmp/openpayments_2023.zip
  inflating: /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt  
  inflating: /tmp/openpayments_2023/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv  


In [0]:
open_payments = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

open_payments.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("General_2023_OpenPayments")

In [0]:
open_payments.columns

Out[3]: ['Change_Type',
 'Covered_Recipient_Type',
 'Teaching_Hospital_CCN',
 'Teaching_Hospital_ID',
 'Teaching_Hospital_Name',
 'Covered_Recipient_Profile_ID',
 'Covered_Recipient_NPI',
 'Covered_Recipient_First_Name',
 'Covered_Recipient_Middle_Name',
 'Covered_Recipient_Last_Name',
 'Covered_Recipient_Name_Suffix',
 'Recipient_Primary_Business_Street_Address_Line1',
 'Recipient_Primary_Business_Street_Address_Line2',
 'Recipient_City',
 'Recipient_State',
 'Recipient_Zip_Code',
 'Recipient_Country',
 'Recipient_Province',
 'Recipient_Postal_Code',
 'Covered_Recipient_Primary_Type_1',
 'Covered_Recipient_Primary_Type_2',
 'Covered_Recipient_Primary_Type_3',
 'Covered_Recipient_Primary_Type_4',
 'Covered_Recipient_Primary_Type_5',
 'Covered_Recipient_Primary_Type_6',
 'Covered_Recipient_Specialty_1',
 'Covered_Recipient_Specialty_2',
 'Covered_Recipient_Specialty_3',
 'Covered_Recipient_Specialty_4',
 'Covered_Recipient_Specialty_5',
 'Covered_Recipient_Specialty_6',
 'Covered_Recipi

In [0]:
# File location and type
file_location = "/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
recipients = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

recipients.columns

Out[4]: ['Covered_Recipient_Profile_Type',
 'Covered_Recipient_Profile_ID',
 'Associated_Covered_Recipient_Profile_ID_1',
 'Associated_Covered_Recipient_Profile_ID_2',
 'Covered_Recipient_NPI',
 'Covered_Recipient_Profile_First_Name',
 'Covered_Recipient_Profile_Middle_Name',
 'Covered_Recipient_Profile_Last_Name',
 'Covered_Recipient_Profile_Suffix',
 'Covered_Recipient_Profile_Alternate_First_Name',
 'Covered_Recipient_Profile_Alternate_Middle_Name',
 'Covered_Recipient_Profile_Alternate_Last_Name',
 'Covered_Recipient_Profile_Alternate_Suffix',
 'Covered_Recipient_Profile_Address_Line_1',
 'Covered_Recipient_Profile_Address_Line_2',
 'Covered_Recipient_Profile_City',
 'Covered_Recipient_Profile_State',
 'Covered_Recipient_Profile_Zipcode',
 'Covered_Recipient_Profile_Country_Name',
 'Covered_Recipient_Profile_Province_Name',
 'Covered_Recipient_Profile_Primary_Specialty',
 'Covered_Recipient_Profile_OPS_Taxonomy_1',
 'Covered_Recipient_Profile_OPS_Taxonomy_2',
 'Covered_Recipient_Pr

In [0]:
from pyspark.sql.functions import col

open_payments.filter(col("Total_Amount_of_Payment_USDollars") > 1000) \
    .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy("count", ascending=False) \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

The payment type “Compensation for services other than consulting” was recorded 164,092 times for amounts above $1,000 while “Consulting Fee” (105,228) and “Travel and Lodging” (24,738) followed as secondary options. The most common payment types after “Compensation for services other than consulting” were “Honoraria” and “Education” and “Royalty or License” and “Grants” with “Debt forgiveness” and “Acquisitions” and “Charitable Contribution” appearing less often but at high values.

In [0]:
open_payments.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(10) \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

Among the overall top ten Nature of Payments “Food and Beverage” maintained the highest count at 13,378,243 while “Travel and Lodging” (545,086) and “Compensation for services other than consulting” (236,628) followed at lower positions. The most common types of payments recorded in the dataset included Consulting Fee and Education and Gift and Honoraria and Royalty or License which demonstrate the wide range of support provided to recipients.

In [0]:
from pyspark.sql.functions import sum as _sum

open_payments.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy("total_amount", ascending=False) \
    .limit(10) \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |total_amount        |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.19217456302E9     |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8 |
|Consulting Fee                                                                                                                                    |5.

The top Nature of Payment category based on dollar value was "Royalty or License" which exceeded $1.19 billion while other categories remained below this amount. The second largest Nature of Payment was "Compensation for services other than consulting" at $594 million and the third was "Consulting Fee" at $515 million. The high-value payment categories also included “Food and Beverage” together with “Travel and Lodging” and “Grants” and “Acquisitions” which demonstrate that although payments happen less often they have significant monetary value.

In [0]:
open_payments.join(recipients, on="Covered_Recipient_Profile_ID", how="inner") \
    .groupBy("Covered_Recipient_Profile_Primary_Specialty") \
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy("total_amount", ascending=False) \
    .limit(10) \
    .show(truncate=False)

+------------------------------------------------------------------------------------------------+-------------------+
|Covered_Recipient_Profile_Primary_Specialty                                                     |total_amount       |
+------------------------------------------------------------------------------------------------+-------------------+
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |3.777555961599989E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.680715036999995E7|
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.587046902999997E7|
|Allopathic & Osteopathic Physicians|Dermatology                                                 |8.076589144000061E7|
|null                                                                                            |7.78133707000005E7 |
|Allopathic & Osteopathic Physicians|Orthopaedic

The medical specialty of “Orthopaedic Surgery” accepted the largest total payments exceeding $377 million which demonstrates robust financial connections between this field and the industry. The highest payments went to “Neurology” and “Neurological Surgery” followed by “Dermatology” while Internal Medicine subspecialties “Hematology & Oncology” and “Cardiovascular Disease” ranked among the ten highest earners. Several records contain insufficient or missing information about specialties as indicated by the occurrence of `null` fields.

In [0]:
from pyspark.sql.functions import concat_ws

open_payments.join(recipients, on="Covered_Recipient_Profile_ID", how="inner") \
    .withColumn("Physician_Name", concat_ws(" ", 
                                            col("Covered_Recipient_Profile_First_Name"),
                                            col("Covered_Recipient_Profile_Middle_Name"),
                                            col("Covered_Recipient_Profile_Last_Name"))) \
    .groupBy("Physician_Name") \
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy("total_amount", ascending=False) \
    .limit(10) \
    .show(truncate=False)

+----------------------+-----------------+
|Physician_Name        |total_amount     |
+----------------------+-----------------+
|STEPHEN S BURKHART    |3.392202493E7    |
|WILLIAM JAY BINDER    |2.943435593E7    |
|KEVIN T FOLEY         |1.730721365E7    |
|IVAN OSORIO           |1.606551551E7    |
|GEORGE PATRICK MAXWELL|1.160032024E7    |
|ROBERT E BOOTH        |8459144.4        |
|NEAL S ELATTRACHE     |7810628.199999999|
|AARON ROSENBERG       |6871466.720000001|
|ROGER P JACKSON       |6660383.800000001|
|PETER M BONUTTI       |6385096.170000002|
+----------------------+-----------------+



The physician with the most significant total payments was Stephen S. Burkhart who received $34 million while William Jay Binder followed with $29.4 million and Kevin T. Foley received $17.3 million. Medical specialists in orthopedics, neurosurgery and internal medicine form the top ten group of recipients who received substantial payments from medical device manufacturers.